# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [2]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

---

In [3]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

### 准备数据

我们使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

下载 zip 文件并将文档解压到 `milvus_docs` 文件夹。

**建议在命令行执行下面命令**

In [4]:
#!wget https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip
#!unzip -q milvus_docs_2.4.x_en.zip -d milvus_docs

我们从 `milvus_docs/en/faq` 文件夹加载所有 markdown 文件。对于每个文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [24]:
from glob import glob

text_lines = []

for file_path in glob("mfd.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [25]:
len(text_lines)

30

### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [26]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [27]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

# from pymilvus import model as milvus_model
#
# # OpenAI国内代理 https://api.apiyi.com/token
# embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
#     model_name='text-embedding-3-large', # Specify the model name
#     api_key='sk-XXX', # Provide your OpenAI API key
#     base_url='https://api.apiyi.com/v1',
#     dimensions=512
# )

生成一个测试嵌入并打印其维度和前几个元素。

In [9]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836061  0.07163018 -0.01130069 -0.03789343 -0.03320646 -0.01318458
 -0.03041719 -0.02269504 -0.02317858 -0.00426026]


In [10]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.02752974  0.06088528  0.00388527 -0.00215194 -0.02774974 -0.01186185
 -0.04020912 -0.06023423 -0.03813158  0.0100272 ]


## 将数据加载到 Milvus

### 创建 Collection

In [28]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"

关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。

In [58]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [59]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [60]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 277768.48it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [61]:
question = "签合同需要注意什么?"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [62]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [63]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4, ensure_ascii=False))

[
    [
        "二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十三条** 以可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权出质的，当事人应当订立书面合同。质权自权利质押登记于相关部门时设立。\n\n**第四百五十四条** 以应收账款出质的，当事人应当订立书面合同。质权自应收账款质押登记于中国人民银行征信中心时设立。\n\n**第四百五十五条** 以法律、行政法规规定可以出质的其他财产权利出质的，依照法律、行政法规的规定。\n\n**第四百五十六条** 权利质权除适用本节规定外，参照适用本章动产质权的有关规定。\n\n####",
        0.6140841245651245
    ],
    [
        "第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [64]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [65]:
context

'二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十三条** 以可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权出质的，当事人应当订立书面合同。质权自权利质押登记于相关部门时设立。\n\n**第四百五十四条** 以应收账款出质的，当事人应当订立书面合同。质权自应收账款质押登记于中国人民银行征信中心时设立。\n\n**第四百五十五条** 以法律、行政法规规定可以出质的其他财产权利出质的，依照法律、行政法规的规定。\n\n**第四百五十六条** 权利质权除适用本节规定外，参照适用本章动产质权的有关规定。\n\n####\n第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意

In [69]:
question

'签合同需要注意什么?'

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [71]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [72]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十三条** 以可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权出质的，当事人应当订立书面合同。质权自权利质押登记于相关部门时设立。\n\n**第四百五十四条** 以应收账款出质的，当事人应当订立书面合同。质权自应收账款质押登记于中国人民银行征信中心时设立。\n\n**第四百五十五条** 以法律、行政法规规定可以出质的其他财产权利出质的，依照法律、行政法规的规定。\n\n**第四百五十六条** 权利质权除适用本节规定外，参照适用本章动产质权的有关规定。\n\n####\n第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [73]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，签合同需要注意以下几点：

1. **合同变更**：合同变更需当事人协商一致，且变更内容必须明确，否则推定为未变更（第五百四十八条、第五百四十九条）。
2. **权利转让**：债权人转让合同权利时，应通知债务人，否则对债务人不发生效力；但根据合同性质、当事人约定或法律规定不得转让的除外（第五百五十条）。
3. **义务转让**：债务人转让合同义务需经债权人同意（第五百五十二条）。
4. **权利和义务一并转让**：需经对方同意，并适用债权和债务转让的相关规定（第五百五十四条、第五百五十五条）。
5. **损害赔偿**：合同变更不影响当事人请求损害赔偿的权利（第五百五十六条）。
6. **地役权合同**：若涉及地役权，合同应明确条款如当事人信息、供役地和需役地位置、地役权目的和期限等，并依法办理登记（第三百八十九条、第三百九十条）。
7. **质权合同**：如涉及权利质权（如汇票、股权、知识产权等），需订立书面合同，并根据权利类型办理交付或登记手续以设立质权（第四百五十条至第四百五十四条）。

总之，签合同时应确保内容明确、遵守转让和变更规定，并依法办理必要登记或交付。

<translated>
根据提供的上下文，签合同需要注意以下几点：

1. **合同变更**：合同变更需当事人协商一致，且变更内容必须明确，否则推定为未变更（第五百四十八条、第五百四十九条）。
2. **权利转让**：债权人转让合同权利时，应通知债务人，否则对债务人不发生效力；但根据合同性质、当事人约定或法律规定不得转让的除外（第五百五十条）。
3. **义务转让**：债务人转让合同义务需经债权人同意（第五百五十二条）。
4. **权利和义务一并转让**：需经对方同意，并适用债权和债务转让的相关规定（第五百五十四条、第五百五十五条）。
5. **损害赔偿**：合同变更不影响当事人请求损害赔偿的权利（第五百五十六条）。
6. **地役权合同**：若涉及地役权，合同应明确条款如当事人信息、供役地和需役地位置、地役权目的和期限等，并依法办理登记（第三百八十九条、第三百九十条）。
7. **质权合同**：如涉及权利质权（如汇票、股权、知识产权等），需订立书面合同，并根据权利类型办理交付或登记手续以设立质权（第四百五十条至第四百五十四条）。

总之，签合同时应确保内容明确、遵守转让和变更规